In [2]:
from bs4 import BeautifulSoup as bs
from datetime import *
from selenium import webdriver
from urllib.request import Request, urlopen
import openpyxl
import pygame
import re
import time

In [4]:
def betweenCarats(test):
    info = []
    while(True):
        start = test.find('>')
        end = test[start:].find('<')
        if end == -1:
            # if the list is empty grab everything before the first <
            if len(info) == 0:
                info.append(test[:start - 3].strip())
            break
        add = test[start + 1:][:end - 1].strip()
        if add != "":
            info.append(add)
        test = test[start + end:]
    return info

In [33]:
NAME        = "A"
PRODUCER    = "B"
DESCRIPTION = "C"
TASTES      = "D"
CITY        = "E"
COUNTRY     = "F"
CULTIVAR    = "G"
PAIRING     = "H"
FIRST_TASTE = 9

# Open an existing file
out = openpyxl.load_workbook('EVOOs2.xlsx')
outsheet = out.worksheets[0]

In [9]:
'''
# Open the worksheet we want to edit
out = openpyxl.Workbook()
outsheet = out.create_sheet("EVOO2s")
# if 'sheet' appears randomly we can delete it
rm = out.get_sheet_by_name('Sheet')
out.remove_sheet(rm)
'''

outsheet[NAME          + '1'].value = "Product"
outsheet[PRODUCER      + '1'].value = "Producer"
outsheet[DESCRIPTION   + '1'].value = "Description"
outsheet[TASTES        + '1'].value = "Tastes"
outsheet[CITY          + '1'].value = "City"
outsheet[COUNTRY       + '1'].value = "Country"
outsheet[CULTIVAR      + '1'].value = "Cultivar"
outsheet[PAIRING       + '1'].value = "Pairing"

# print(len(innerHTML))
url = 'https://bestoliveoils.com'
brands = '/brands/'
producers = '/producers/'

field0 = 'producer-description'
field1 = 'Tasting Sensations'
field2 = 'Region'
field3 = 'Cultivar'
field4 = 'Food pairing'

text = open("list.txt", "r").read()
evoos = text.splitlines()
info = []

for j in range(200, len(evoos)):
    browser = webdriver.Chrome()

    brand = evoos[j]

    fullUrl = url + brands + brand

    print(fullUrl)

    browser.get(fullUrl)
    HTML = browser.execute_script("return document.body.innerHTML")
    innerHTML = HTML.split('\n')
    evoo = {}
    for i in range(0, len(innerHTML)):
        if field0 in innerHTML[i]:
            evoo['description'] = innerHTML[i + 1]
            evoo['producer']    = betweenCarats(innerHTML[i- 3])
        if field1 in innerHTML[i]:
            for line in range(i + 2, len(innerHTML), 2):
                if 'd-inline' in innerHTML[line]:
                    temp = ""
                    if field1 in evoo:
                        temp = evoo[field1]
                    evoo[field1] = temp + betweenCarats(innerHTML[line])[0] + ", "         
                else:
                    evoo[field1] = evoo[field1][:-2]
                    break
        if field2 in innerHTML[i]:
            for line in range(i + 1, len(innerHTML)):
                if '</a>' in innerHTML[line]:
                    temp = ""
                    if field2 in evoo:
                        temp = evoo[field2]
                    evoo[field2] = temp + betweenCarats(innerHTML[line])[0] + ", "
                if field3 in innerHTML[line]:
                    evoo[field2] = evoo[field2][:-2]
                    break
        if field3 in innerHTML[i]:
            evoo[field3] = betweenCarats(innerHTML[i + 2])
        if field4 in innerHTML[i]:
            evoo[field4] = betweenCarats(innerHTML[i + 2])
            break
            
    print("Now putting info into row", str(j + 2), evoo)
    
    outsheet[NAME + str(j + 2)].value = brand.replace('-', ' ').strip().title()
    if 'producer' in evoo:
        outsheet[PRODUCER      + str(j + 2)].value = evoo["producer"][0].strip()
    if 'description' in evoo:
        outsheet[DESCRIPTION   + str(j + 2)].value = evoo["description"].strip()
    if 'Tasting Sensations' in evoo:
        outsheet[TASTES        + str(j + 2)].value = evoo["Tasting Sensations"].strip()
    if 'Region' in evoo:
        where = evoo["Region"].split(', ')
        if len(where) == 2:
            outsheet[CITY    + str(j + 2)].value = where[0]
            outsheet[COUNTRY + str(j + 2)].value = where[1]
        else:
            outsheet[COUNTRY + str(j + 2)].value = where[0]
    if 'Cultivar' in evoo:
        outsheet[CULTIVAR      + str(j + 2)].value = evoo["Cultivar"][0].strip()
    if 'Food pairing' in evoo:
        pairing = ""
        for i in range(0, len(evoo['Food pairing'])):
            pairing = pairing + evoo['Food pairing'][i] + ": "
        outsheet[PAIRING       + str(j + 2)].value = pairing[:-2].strip()
    out.save("EVOOs2.xlsx")
    
    info.append(evoo)
    browser.quit()

#print(info)

https://bestoliveoils.com/brands/liofos
Now putting info into row 202 {'Cultivar': ['Blend'], 'description': '      Stamatakou Elaionas is a family business that deals entirely with the production and disposal of its manufactured products. It is an individual vertically integrated biological unit that includes: the ', 'producer': ['Stamatakou Elaionas'], 'Tasting Sensations': 'Flowers, Mint', 'Region': 'Lakonia, Greece', 'Food pairing': ['Chocolate Ice Cream', 'Match: 4.7 / 5']}
https://bestoliveoils.com/brands/lodi-frantoio
Now putting info into row 203 {'Cultivar': ['Frantoio'], 'description': '      Calivirgin is an Italian family-owned and operated company that has been farming in the San Joaquin Valley of California for more than 80 years. We are committed to producing the highest quality extra virgin olive oil on the market today.', 'producer': ['Calivirgin Coldani Olive Ranch'], 'Tasting Sensations': 'Almond, Tomato, Almond, Ripe Olives, Grass', 'Region': 'California, United Sta

In [42]:
first = 2
last = outsheet.max_row + 1
noTastes = 0
tasty = {}
for row in range (first, last + 1):
    tastes = outsheet[TASTES + str(row)].value
    print(row, tastes)
    if tastes:
        tasteList = tastes.split(', ')
        uniques = []
        for i in tasteList:
            if i not in uniques:
                uniques.append(i)
        outsheet[TASTES + str(row)].value = ", ".join(uniques)
        for i in uniques:
            if i not in tasty:
                tasty[i] = 1
            else:
                tasty[i] = tasty [i] + 1
    else:
        noTastes = noTastes + 1
    
# sort the dictionary
sorted_d = sorted((value, key) for (key,value) in tasty.items())
sorted_d = list(reversed(sorted_d))
print(sorted_d)
print("Found", len(tasty), "different tastes")
print(noTastes, "olive oils had no tastes listed")

2 Spicy, Banana, Bitter Almond, Arugula, Green Banana
3 Artichoke, Herbs, Bitter Almond, Wheat, Tomato Leaf
4 Olive Leaf, Vegetable, Tomato Leaf, Tomato, Green Tomato
5 Almond, Vanilla, Balsamic, Almond, Bitter Almond
6 Green Tea, Leaf, Berry
7 Artichoke, Almond, Arugula, Apple, Artichoke
8 Celery, Arugula, Grass, Almond, Arugula
9 Pine Seed, Red Chili Pepper, Balsamic, Black Pepper, Herbs
10 Artichoke, Tomato Leaf, Arugula, Artichoke
11 Artichoke, Green Pepper, Bitter Almond, Mint, Red Chili Pepper
12 Green Fruit, Tomato, Olive Leaf, Green Tomato, Green Fruit
13 Chicory, Arugula, Black Pepper, Chicory, Arugula
14 Green Almond, Flowers, Pepper
15 Almond, Walnut, Green Pepper, Pink Pepper, Spicy
16 None
17 Ripe Olives, Ripe Notes, Ripe Fruit, Tomato, Grass
18 Bitter Almond, Arugula, Leaf, Almond, Walnut
19 Almond, Banana, Green Almond, Green Tomato, Green Apple
20 Chamomile, Chicory, Black Pepper, Eucalyptus, Almond
21 Almond, Banana, Flowers, Cinnamon, Fennel
22 Fig Leaf, Almond, Chico

In [43]:
# HOW TO GET A COLUMN BY NUMBER
# for each match we need to create headers and populate columns
tasteHeaders = {}
for i in range(0, len(sorted_d)):
    num, header = sorted_d[i]
    print(num, header)
    col = FIRST_TASTE + i
    outsheet.cell(row=1,column=col).value = header
    tasteHeaders[header] = col
    
out.save("EVOOs2.xlsx")

183 Artichoke
149 Almond
99 Herbs
86 Green Almond
79 Arugula
62 Bitter Almond
45 Green Pepper
42 Grass
41 Walnut
39 Green Tomato
39 Flowers
37 Olive Leaf
37 Fig Leaf
36 Tomato Leaf
33 Chicory
33 Black Pepper
28 Apple
27 Green Banana
25 Green Fruit
24 Green Tea
24 Banana
23 Leaf
19 Radish
18 Tomato
17 Pepper
16 Mint
15 Vanilla
15 Cinnamon
14 Balsamic
13 Spicy
13 Green Apple
11 Celery
10 Pine Seed
10 Eucalyptus
10 Chamomile
9 Red Chili Pepper
9 Pink Pepper
9 Fennel
9 Dry Fruit
9 Basil
8 Unripe
8 Pine Nuts
8 Exotic Fruits
8 Butter
7 Ripe Notes
7 Ripe Fruit
7 Nuts
7 Apricot
6 Hot Chili
5 Spice
5 Rosemary
5 Ripe Olives
5 Golden Apple
4 Red Pepper
4 Lettuce
4 Citrus
4 Black Olives
4 Anise (Pimpinella)
3 Wheat
3 Vegetable
3 Thyme
3 Tea
3 Sage
3 Peppermint
3 Oregano
3 Cherry
3 Berry
2 Red Fruits
2 Peach
2 Passion Fruit
2 Parsley
2 Mango
2 Cucumber
2 Coffee
2 Cocoa
1 Ripe Apple
1 Pear
1 Orange Zest
1 Garlic


In [44]:
first = 2
last = outsheet.max_row + 1
tasty = {}
for row in range (first, last + 1):
    tastes = outsheet[TASTES + str(row)].value
    if tastes:
        tasteList = tastes.split(', ')
        for i in tasteList:
            outsheet.cell(row=row,column=tasteHeaders[i]).value = "Present"

out.save("EVOOs2.xlsx")